# TCS K7 Metric Construction

**GPU-accelerated computation for Twisted Connected Sum G2 manifolds**

This notebook implements:
- Phase 3: Explicit G2 3-form φ_L on the TCS neck
- Phase 5: Metric extraction and normalization
- Phase 6: Spectral gap computation via finite elements

Requires: Colab Pro+ with A100 GPU for large-scale computations.

In [ ]:
# Environment setup
import numpy as np
import json
from typing import Tuple, Callable

# GPU detection
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print(f"GPU available: {cp.cuda.runtime.getDeviceCount()} device(s)")
    xp = cp  # Use CuPy
except ImportError:
    GPU_AVAILABLE = False
    print("No GPU - using NumPy")
    xp = np  # Fallback to NumPy

# Constants from TCS construction
B2_M1 = 11  # Quintic
B3_M1 = 40
B2_M2 = 10  # CI(2,2,2)
B3_M2 = 37

B2_K7 = B2_M1 + B2_M2  # = 21
B3_K7 = B3_M1 + B3_M2  # = 77
H_STAR = 1 + B2_K7 + B3_K7  # = 99
DIM_G2 = 14

print(f"K7 Betti numbers: b2={B2_K7}, b3={B3_K7}")
print(f"H* = {H_STAR}")

## 1. G2 Structure Constants

The standard G2 3-form in orthonormal frame {e¹,...,e⁷}:
$$\varphi = e^{127} + e^{347} + e^{567} + e^{135} - e^{146} - e^{236} - e^{245}$$

In [ ]:
def g2_structure_constants() -> np.ndarray:
    """
    Return the G2 structure constants φ_ijk.
    
    Uses Fano plane convention (1-indexed internally, 0-indexed output).
    """
    phi = np.zeros((7, 7, 7))
    
    # Positive terms (from Fano plane lines)
    positive_triples = [
        (0, 1, 6),  # e^127
        (2, 3, 6),  # e^347
        (4, 5, 6),  # e^567
        (0, 2, 4),  # e^135
    ]
    
    # Negative terms
    negative_triples = [
        (0, 3, 5),  # e^146
        (1, 2, 5),  # e^236
        (1, 3, 4),  # e^245
    ]
    
    # Fill in with total antisymmetry
    from itertools import permutations
    
    def sign_of_perm(p, base):
        """Compute sign of permutation"""
        n = len(p)
        sign = 1
        for i in range(n):
            for j in range(i+1, n):
                if base.index(p[i]) > base.index(p[j]):
                    sign *= -1
        return sign
    
    for i, j, k in positive_triples:
        base = (i, j, k)
        for p in permutations(base):
            s = sign_of_perm(p, base)
            phi[p[0], p[1], p[2]] = s
    
    for i, j, k in negative_triples:
        base = (i, j, k)
        for p in permutations(base):
            s = sign_of_perm(p, base)
            phi[p[0], p[1], p[2]] = -s
    
    return phi

PHI = g2_structure_constants()
print(f"G2 structure tensor shape: {PHI.shape}")
print(f"Non-zero entries: {np.count_nonzero(PHI)}")
print(f"Should be 7 × 6 = 42 (7 terms × 3! permutations)")

## 2. TCS Neck Geometry

The neck region is T² × K3 × I where I = [0, L].

We discretize using a product grid.

In [ ]:
class TCSNeck:
    """
    Discretized TCS neck geometry.
    
    Coordinates:
    - psi ∈ [0, 2π): extra S¹
    - r ∈ [0, L]: radial (neck direction)
    - theta ∈ [0, 2π): fiber S¹
    - K3 approximated as T⁴ for computation
    """
    
    def __init__(self, L: float, N_r: int = 50, N_angle: int = 16, N_K3: int = 8):
        """
        Initialize TCS neck.
        
        Args:
            L: Neck length
            N_r: Grid points in radial direction
            N_angle: Grid points for each S¹
            N_K3: Grid points per K3 direction (using T⁴ approximation)
        """
        self.L = L
        self.N_r = N_r
        self.N_angle = N_angle
        self.N_K3 = N_K3
        
        # Grid spacing
        self.dr = L / (N_r - 1)
        self.dpsi = 2 * np.pi / N_angle
        self.dtheta = 2 * np.pi / N_angle
        self.dK3 = 2 * np.pi / N_K3  # For T⁴ approximation
        
        # Total grid size (reduced for memory)
        self.N_total = N_r * N_angle * N_angle  # Ignoring K3 for now
        
        print(f"TCS Neck: L={L}, grid size={self.N_total}")
    
    def metric_at_point(self, psi: float, r: float, theta: float) -> np.ndarray:
        """
        Return 7×7 metric tensor at a point.
        
        On the neck, metric is product:
        ds² = dpsi² + dr² + dtheta² + ds²_K3
        """
        g = np.eye(7)
        # Product metric is just identity in orthonormal frame
        return g
    
    def volume(self) -> float:
        """
        Approximate volume of the neck region.
        
        Vol(T² × K3 × I) = (2π)² × Vol(K3) × L
        """
        Vol_K3 = 16 * np.pi**2  # Topological normalization
        Vol_T2 = (2 * np.pi)**2
        return Vol_T2 * Vol_K3 * self.L

# Test
neck = TCSNeck(L=10.0, N_r=50)
print(f"Volume: {neck.volume():.2f}")

## 3. Laplacian on the Neck

For the product metric, the Laplacian separates:
$$\Delta = \Delta_{T^2} + \partial_r^2 + \Delta_{K3}$$

We focus on the r-direction for the spectral gap.

In [ ]:
def build_1d_laplacian_sparse(N: int, h: float, periodic: bool = False):
    """
    Build 1D Laplacian as sparse matrix.
    
    CuPy-compatible: builds COO directly, converts to CSR.
    
    Args:
        N: Number of grid points
        h: Grid spacing
        periodic: If True, use periodic boundary conditions
    
    Returns:
        Sparse CSR matrix (CuPy or SciPy depending on GPU_AVAILABLE)
    """
    row, col, data = [], [], []
    
    for i in range(N):
        # Diagonal: -2/h²
        row.append(i)
        col.append(i)
        data.append(-2.0 / h**2)
        
        # Off-diagonals: 1/h²
        if i > 0:
            row.append(i)
            col.append(i - 1)
            data.append(1.0 / h**2)
        elif periodic:
            row.append(i)
            col.append(N - 1)
            data.append(1.0 / h**2)
        
        if i < N - 1:
            row.append(i)
            col.append(i + 1)
            data.append(1.0 / h**2)
        elif periodic:
            row.append(i)
            col.append(0)
            data.append(1.0 / h**2)
    
    if GPU_AVAILABLE:
        return cp_csr(
            (cp.array(data), (cp.array(row), cp.array(col))),
            shape=(N, N)
        )
    else:
        from scipy.sparse import csr_matrix
        return csr_matrix(
            (np.array(data), (np.array(row), np.array(col))),
            shape=(N, N)
        )

# Test: 1D Laplacian on [0, L] with Dirichlet BC
L_test = 10.0
N_test = 100
h_test = L_test / (N_test - 1)
D2 = build_1d_laplacian_sparse(N_test, h_test, periodic=False)
print(f"Laplacian shape: {D2.shape}")

## 4. Spectral Gap Computation

For the r-direction Laplacian on [0, L] with Dirichlet BC:
$$\lambda_n = \frac{n^2 \pi^2}{L^2}$$

So λ₁ = π²/L².

In [ ]:
def compute_spectral_gap(L: float, N: int = 200, k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute lowest eigenvalues of 1D Laplacian on [0, L].
    
    Args:
        L: Interval length
        N: Number of grid points
        k: Number of eigenvalues to compute
    
    Returns:
        eigenvalues, eigenvectors
    """
    h = L / (N - 1)
    
    # Build negative Laplacian (so eigenvalues are positive)
    D2 = build_1d_laplacian_sparse(N, h, periodic=False)
    negD2 = -D2
    
    # Compute smallest algebraic eigenvalues
    # For CuPy: use 'SA' (smallest algebraic), not 'SM'
    if GPU_AVAILABLE:
        eigenvalues, eigenvectors = cp_eigsh(negD2, k=k, which='SA')
        eigenvalues = cp.asnumpy(eigenvalues)
        eigenvectors = cp.asnumpy(eigenvectors)
    else:
        from scipy.sparse.linalg import eigsh
        eigenvalues, eigenvectors = eigsh(negD2, k=k, which='SA')
    
    # Sort by magnitude
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    return eigenvalues, eigenvectors

# Test for various L
print("Spectral gap vs neck length:")
print("-" * 40)
print(f"{'L':>8} {'λ₁ (num)':>12} {'λ₁ (exact)':>12} {'Error %':>10}")
print("-" * 40)

for L in [5.0, 10.0, np.sqrt(H_STAR), 15.0, 20.0]:
    eigs, _ = compute_spectral_gap(L, N=200, k=3)
    lambda1_num = eigs[0]
    lambda1_exact = np.pi**2 / L**2
    error = abs(lambda1_num - lambda1_exact) / lambda1_exact * 100
    print(f"{L:8.2f} {lambda1_num:12.6f} {lambda1_exact:12.6f} {error:10.4f}")

print("-" * 40)
print(f"\nNote: L = √H* = √99 ≈ {np.sqrt(H_STAR):.3f}")

## 5. GIFT Selection Principle Test

Test if L² = H* gives λ₁ = dim(G₂)/H*

In [ ]:
# GIFT predictions
GIFT_LAMBDA1 = DIM_G2 / H_STAR  # = 14/99
GIFT_LAMBDA1_ALT = 8 / H_STAR   # = 8/99 (alternative from r₃²)

print("GIFT Spectral Gap Predictions:")
print(f"  λ₁ = dim(G₂)/H* = {DIM_G2}/{H_STAR} = {GIFT_LAMBDA1:.6f}")
print(f"  λ₁ = 8/H* = 8/{H_STAR} = {GIFT_LAMBDA1_ALT:.6f}")
print()

# What L gives these values?
def L_for_lambda1(lam1):
    """Invert λ₁ = π²/L² to get L"""
    return np.pi / np.sqrt(lam1)

L_gift = L_for_lambda1(GIFT_LAMBDA1)
L_gift_alt = L_for_lambda1(GIFT_LAMBDA1_ALT)

print(f"Required L for λ₁ = {GIFT_LAMBDA1:.4f}: L = {L_gift:.4f}")
print(f"  L² = {L_gift**2:.4f}")
print(f"  H* = {H_STAR}")
print(f"  Ratio L²/H* = {L_gift**2 / H_STAR:.4f}")
print()
print(f"Required L for λ₁ = {GIFT_LAMBDA1_ALT:.4f}: L = {L_gift_alt:.4f}")
print(f"  L² = {L_gift_alt**2:.4f}")
print(f"  Ratio L²/H* = {L_gift_alt**2 / H_STAR:.4f}")
print()

# Check if L² ∝ H*
print("Selection principle check:")
print(f"  If L² = H* = 99: λ₁ = π²/99 = {np.pi**2/99:.6f}")
print(f"  GIFT predicts: λ₁ = 14/99 = {14/99:.6f}")
print(f"  Ratio: (14/99) / (π²/99) = 14/π² = {14/np.pi**2:.4f}")

## 6. Full TCS Spectral Analysis

The full spectral gap includes contributions from K3 × T².

In [ ]:
def tcs_spectral_gap(L: float, include_K3: bool = False) -> dict:
    """
    Compute TCS spectral gap with various contributions.
    
    Returns dictionary with:
    - lambda1_r: Contribution from r-direction
    - lambda1_T2: First non-zero mode from T² (if computed)
    - lambda1_K3: First non-zero mode from K3 (if computed)
    - lambda1_total: Combined spectral gap
    """
    results = {}
    
    # r-direction (dominant for large L)
    results['lambda1_r'] = float(np.pi**2 / L**2)
    
    # T² contribution: first non-zero mode has eigenvalue 1 (for unit radius)
    # On T² with radii (1, 1): λ = n² + m², first non-zero is 1
    results['lambda1_T2'] = 1.0
    
    # K3 contribution: spectral gap of Ricci-flat K3
    # This depends on the K3 moduli; for generic K3, λ₁(K3) ≈ 0.1-1
    # We use a placeholder
    results['lambda1_K3'] = 0.5 if include_K3 else float('inf')
    
    # Total: minimum of all directions
    results['lambda1_total'] = min(
        results['lambda1_r'],
        results['lambda1_T2'],
        results['lambda1_K3']
    )
    
    # Bounds from Model Theorem
    v0 = 0.5  # Symmetric blocks
    v1 = 0.5
    results['lower_bound'] = float(v0**2 / L**2)
    results['upper_bound'] = float(16 * v1 / ((1 - v1) * L**2))
    
    return results

# Scan L values
L_values = np.linspace(5, 20, 16)
results_list = []

for L in L_values:
    res = tcs_spectral_gap(L)
    res['L'] = float(L)
    res['L_squared'] = float(L**2)
    results_list.append(res)

# Find L where λ₁ matches GIFT prediction
print("\nSearching for L matching GIFT predictions...")
for target_name, target_val in [('14/99', 14/99), ('8/99', 8/99)]:
    # Find L where λ₁_r = target
    L_match = np.pi / np.sqrt(target_val)
    print(f"\nFor λ₁ = {target_name} = {target_val:.6f}:")
    print(f"  Required L = {L_match:.4f}")
    print(f"  L² = {L_match**2:.4f}")
    print(f"  L²/H* = {L_match**2/H_STAR:.4f}")

## 7. Export Results

In [ ]:
# Compile final results
final_results = {
    'building_blocks': {
        'M1_quintic': {'b2': int(B2_M1), 'b3': int(B3_M1)},
        'M2_CI': {'b2': int(B2_M2), 'b3': int(B3_M2)}
    },
    'K7_topology': {
        'b2': int(B2_K7),
        'b3': int(B3_K7),
        'H_star': int(H_STAR),
        'dim_G2': int(DIM_G2)
    },
    'spectral_predictions': {
        'GIFT_lambda1_primary': float(14/99),
        'GIFT_lambda1_alternate': float(8/99),
        'TCS_L_for_primary': float(np.pi / np.sqrt(14/99)),
        'TCS_L_for_alternate': float(np.pi / np.sqrt(8/99))
    },
    'model_theorem': {
        'v0': 0.5,
        'v1': 0.5,
        'scaling': 'lambda1 ~ 1/L^2'
    },
    'selection_principle': {
        'status': 'CONJECTURAL',
        'formula': 'L^2 ~ kappa * H*',
        'kappa_for_14_99': float((np.pi / np.sqrt(14/99))**2 / H_STAR)
    }
}

# Save to JSON
with open('tcs_k7_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("Results saved to tcs_k7_results.json")
print(json.dumps(final_results, indent=2))

## 8. Summary

### Key Findings

1. **TCS gives λ₁ ~ π²/L²** for the radial direction
2. **GIFT predicts λ₁ = 14/99** which requires L ≈ 8.35
3. **Selection principle**: L² ≈ 0.70 × H* = 70 to match GIFT

### Open Questions

- What mechanism selects L?
- Is the coefficient 14/π² ≈ 1.42 universal?
- How does K3 moduli affect the spectral gap?